In [1]:
import sys, os, time

sys.path.append("../..")
import pyzx as zx
from benchmarking import Benchmark
import numpy as np
import argparse
import json
import random
import time
from distutils.util import strtobool
from typing import Optional

import gymnasium as gym
import gym_zx
import numpy as np
import pyzx as zx

def parse_args():
    # fmt: off
    parser = argparse.ArgumentParser()
    parser.add_argument("--exp-name", type=str, default=os.path.basename(sys.argv[0]).rstrip(".py"),
                        help="the name of this experiment")
    parser.add_argument("--gym-id", type=str, default="zx-v0",
                        help="the id of the gym environment")
    parser.add_argument("--learning-rate", type=float, default=2e-4,
                        help="the learning rate of the optimizer")
    parser.add_argument("--seed", type=int, default=8983440,
                        help="seed of the experiment")
    parser.add_argument("--total-timesteps", type=int, default=10000000,
                        help="total timesteps of the experiments")
    parser.add_argument("--torch-deterministic", type=lambda x: bool(strtobool(x)), default=True, nargs="?", const=True,
                        help="if toggled, `torch.backends.cudnn.deterministic=False`")
    parser.add_argument("--cuda", type=lambda x: bool(strtobool(x)), default=True, nargs="?", const=True,
                        help="if toggled, cuda will be enabled by default")
    parser.add_argument("--track", type=lambda x: bool(strtobool(x)), default=False, nargs="?", const=True,
                        help="if toggled, this experiment will be tracked with Weights and Biases")
    parser.add_argument("--wandb-project-name", type=str, default="ppo-implementation-details",
                        help="the wandb's project name")
    parser.add_argument("--wandb-entity", type=str, default=None,
                        help="the entity (team) of wandb's project")
    parser.add_argument("--capture-video", type=lambda x: bool(strtobool(x)), default=False, nargs="?", const=True,
                        help="weather to capture videos of the agent performances (check out `videos` folder)")
    # Algorithm specific arguments
    parser.add_argument("--num-envs", type=int, default=4,
                        help="the number of parallel game environments") #default 8
    parser.add_argument("--num-steps", type=int, default=2048,
                        help="the number of steps to run in each environment per policy rollout")
    parser.add_argument("--anneal-lr", type=lambda x: bool(strtobool(x)), default=True, nargs="?", const=True,
                        help="Toggle learning rate annealing for policy and value networks")
    parser.add_argument("--gae", type=lambda x: bool(strtobool(x)), default=True, nargs="?", const=True,
                        help="Use GAE for advantage computation")
    parser.add_argument("--gamma", type=float, default=0.99,
                        help="the discount factor gamma")
    parser.add_argument("--gae-lambda", type=float, default=0.95,
                        help="the lambda for the general advantage estimation")
    parser.add_argument("--num-minibatches", type=int, default=16,
                        help="the number of mini-batches")
    parser.add_argument("--update-epochs", type=int, default=8,
                        help="the K epochs to update the policy")
    parser.add_argument("--norm-adv", type=lambda x: bool(strtobool(x)), default=True, nargs="?", const=True,
                        help="Toggles advantages normalization")
    parser.add_argument("--clip-coef", type=float, default=0.1,
                        help="the surrogate clipping coefficient")
    parser.add_argument("--clip-vloss", type=lambda x: bool(strtobool(x)), default=True, nargs="?", const=True,
                        help="Toggles whether or not to use a clipped loss for the value function, as per the paper.")
    parser.add_argument("--ent-coef", type=float, default=0.05,
                        help="coefficient of the entropy")
    parser.add_argument("--vf-coef", type=float, default=0.5,
                        help="coefficient of the value function")
    parser.add_argument("--max-grad-norm", type=float, default=0.5,
                        help="the maximum norm for the gradient clipping")
    parser.add_argument("--target-kl", type=float, default=None,
                        help="the target KL divergence threshold")

    if sys.argv[0].endswith('ipykernel_launcher.py'):
        # This script is being run in a Jupyter notebook
        args = parser.parse_args([])
    else:
        args = parser.parse_args()

    args.batch_size = int(args.num_envs * args.num_steps)
    args.minibatch_size = int(args.batch_size // args.num_minibatches)
    # fmt: on
    return args

In [2]:
b = Benchmark()

In [8]:
b.load_circuits(
    dirname="/home/jnogue/qilimanjaro/Copt-cquere/src/pyzx/circuits/benchmarking_circuits/Fast/before/Arithmetic_and_Toffoli",
    group_name="fast",
)

b.load_circuits(
    dirname=os.path.join("..", "..","..", "pyzx", "circuits", "benchmarking_circuits", "Fast", "nrscm", "nrscm_noQFT"),
    group_name="fast",
    simp_strategy="NRSCM",
)
b.load_circuits(
    dirname=os.path.join("..", "..","..", "pyzx", "circuits", "benchmarking_circuits", "Fast", "tpar", "tpar_noQFT"),
    group_name="fast",
    simp_strategy="TPar",
)
b.show_attributes()

Circuit attributes:  ['Qubits', 'Gates', '2Q Count', 'T Count', 't_opt']
Loaded functions:  ['flow-opt']
Loaded routines:  ['NRSCM', 'TPar']
Loaded circuit groups:  ['fast']


,Original,NRSCM,TPar,flow-opt
fast,Y,Y,-,Y


In [2]:
def basic_optimise(c):
    c1 = zx.basic_optimization(c.copy(), do_swaps=False).to_basic_gates()
    c2 = zx.basic_optimization(c.copy(), do_swaps=True).to_basic_gates()
    if c2.twoqubitcount() < c1.twoqubitcount():
        return c2  # As this optimisation algorithm is targetted at reducting H-gates, we use the circuit with the smaller 2-qubit gate count here, either using SWAP rules or not.
    return c1


def flow_opt(c):
    g = c.to_graph()
    zx.teleport_reduce(g)
    zx.to_graph_like(g)
    zx.flow_2Q_simp(g)
    c2 = zx.extract_simple(g).to_basic_gates()
    return basic_optimise(c2)

In [5]:
import torch
from torch_geometric.data import Batch, Data
import torch.multiprocessing as mp
import sys
sys.path.append('/home/jnogue/qilimanjaro/Copt-cquere/rl-zx/')
from rl_agent import AgentGNN


def rl_zx(c):
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    best_result = np.inf
    stats = c.stats_dict()
    qubits = stats["qubits"]
    depth = stats["depth"]
    args = parse_args()
    envs = gym.make("zx-v0", qubits=qubits, depth=depth, circuit=c)
    """envs = gym.vector.SyncVectorEnv(
        [make_env(args.gym_id, args.seed + i, i, True, "Benchmarking", qubits=qubits, gates=depth, circuit=c) for i in range(args.num_envs)]
    )"""

    agent = AgentGNN(envs, device).to(device)  # Pass the envs argument here

    agent.load_state_dict(
        torch.load("/home/jnogue/qilimanjaro/Copt-cquere/rl-zx/state_dict_5x70_cquere_twoqubits.pt", map_location=torch.device("cpu"))
    )  # Add the map_location argument here
    agent.eval()

  

    for episode in range(1):
  
        obs0, reset_info = envs.reset()
      
        policy_items, value_items = reset_info["graph_obs"]
        value_graph = [Data(x=value_items[0], edge_index=value_items[1], edge_attr=value_items[2])]
        policy_graph = [Data(x=policy_items[0], edge_index=policy_items[1], edge_attr=policy_items[2], y=policy_items[3])]
        next_obs_graph = (Batch.from_data_list(policy_graph), Batch.from_data_list(value_graph))
        done = False
        while not done:
            action, action_ids = agent.get_action_and_value(next_obs_graph, device=device, testing=True)
            next_obs, reward, done, deprecated, info = envs.step(action_ids.cpu().numpy())
            policy_items, value_items = info["graph_obs"]
            value_graph = [Data(x=value_items[0], edge_index=value_items[1], edge_attr=value_items[2])]
            policy_graph = [Data(x=policy_items[0], edge_index=policy_items[1], edge_attr=policy_items[2], y=policy_items[3])]
            next_obs_graph = (Batch.from_data_list(policy_graph), Batch.from_data_list(value_graph))

      
        twoq_gates = info["rl_stats"]["twoqubits"]
        if twoq_gates < best_result:
            circuit = info["final_circuit"]
            best_result = twoq_gates
    return circuit#, action_seq, graph

In [11]:
import os
import pyzx as zx

dirname = os.path.join("..", "..","..", "pyzx", "circuits", "benchmarking_circuits", "Fast", "before", "before_noQFT")

# Iterate over all files in the directory
# Iterate over all files in the directory
for filename in os.listdir(dirname):
    file_path = os.path.join(dirname, filename)
    if os.path.isfile(file_path):  # Check if it is a file
        try:
            circuit = zx.Circuit.load(file_path).to_basic_gates()
            opt_circuit = rl_zx(circuit)
            print(opt_circuit)
            # Process the circuit here
            print(f"Processed circuit from file: {filename}")
        except Exception as e:
            print(f"Error processing file {filename}: {e}")


/home/jnogue/miniconda3/envs/cquere/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:159: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
/home/jnogue/miniconda3/envs/cquere/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:159: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


Win vs Pyzx:  0  Episode Gates:  24 Cflow_gates:  24 Episode Len 26 Opt Episode Len 0 episode 1
Circuit(5 qubits, 0 bits, 56 gates)
Processed circuit from file: barenco_tof_3
Win vs Pyzx:  0  Episode Gates:  42 Cflow_gates:  42 Episode Len 11 Opt Episode Len 0 episode 1
Circuit(9 qubits, 0 bits, 102 gates)
Processed circuit from file: tof_5
Win vs Pyzx:  0  Episode Gates:  192 Cflow_gates:  192 Episode Len 50 Opt Episode Len 0 episode 1
Circuit(19 qubits, 0 bits, 427 gates)
Processed circuit from file: barenco_tof_10
Win vs Pyzx:  -1  Episode Gates:  385 Cflow_gates:  383 Episode Len 50 Opt Episode Len 0 episode 1
Circuit(24 qubits, 0 bits, 822 gates)
Processed circuit from file: adder_8
Win vs Pyzx:  0  Episode Gates:  99 Cflow_gates:  99 Episode Len 50 Opt Episode Len 0 episode 1
Circuit(12 qubits, 0 bits, 207 gates)
Processed circuit from file: gf2^4_mult
Win vs Pyzx:  0  Episode Gates:  18 Cflow_gates:  18 Episode Len 18 Opt Episode Len 0 episode 1
Circuit(5 qubits, 0 bits, 44 gate

In [12]:
b.add_simplification_func(func=rl_zx, name="RL-ZX", groups_to_run=None, verify=False, rerun=False)
#b.add_simplification_func(func=flow_opt, name="flow-opt", groups_to_run=["fast"], verify=False, rerun=False)


b.run(funcs_to_run=['all'], groups_to_run=['fast'], verify=True, rerun=False)
#b.run(funcs_to_run=["RL-ZX"], groups_to_run=["fast"], verify=True, rerun=False)

b.show_attributes()

Processing RL-ZX on tof_5                                             :   4%|▎         | 1/28 [00:00<00:21,  1.29it/s]

Win vs Pyzx:  0  Episode Gates:  24 Cflow_gates:  24 Episode Len 14 Opt Episode Len 0 episode 1


Processing RL-ZX on barenco_tof_10                                    :   7%|▋         | 2/28 [00:04<01:01,  2.38s/it]

Win vs Pyzx:  0  Episode Gates:  42 Cflow_gates:  42 Episode Len 40 Opt Episode Len 0 episode 1
Win vs Pyzx:  0  Episode Gates:  192 Cflow_gates:  192 Episode Len 50 Opt Episode Len 0 episode 1


Processing RL-ZX on adder_8                                           :  11%|█         | 3/28 [00:50<09:19, 22.39s/it]

...

Processing RL-ZX on gf2^7_mult                                        :  96%|█████████▋| 27/28 [16:27<00:46, 46.51s/it]

Win vs Pyzx:  0  Episode Gates:  300 Cflow_gates:  300 Episode Len 50 Opt Episode Len 0 episode 1


Processing RL-ZX on gf2^7_mult                                        : 100%|██████████| 28/28 [17:36<00:00, 37.73s/it]

Circuit attributes:  ['Qubits', 'Gates', '2Q Count', 'T Count', 't_opt']
Loaded functions:  ['RL-ZX']
Loaded routines:  ['NRSCM', 'TPar']
Loaded circuit groups:  ['fast']


,Original,NRSCM,RL-ZX,TPar
fast,Y,Y,Y,-


In [1]:
b.add_simplification_func(func=flow_opt, name="flow-opt", groups_to_run=["fast"], verify=False, rerun=False)
b.run(funcs_to_run=["flow-opt"], groups_to_run=["fast"], verify=True, rerun=False)

NameError: name 'b' is not defined

In [10]:
df = b.df(groups=["fast"], routines=["all"], funcs=["all"], atts=["Qubits","Gates","2Q Count","T Count"])

In [15]:
b.show_attributes()

Circuit attributes:  ['Qubits', 'Gates', '2Q Count', 'T Count', 't_opt']
Loaded functions:  ['RL-ZX']
Loaded routines:  ['NRSCM', 'TPar']
Loaded circuit groups:  ['fast']


,Original,NRSCM,RL-ZX,TPar
fast,Y,Y,Y,-


In [16]:
df = b.df(groups=["fast"], routines=["all"], funcs=["all"], atts=["Qubits", "Gates", "2Q Count"])

In [3]:
import pyzx as zx
circ = zx.Circuit.from_qasm_file("/home/jnogue/qilimanjaro/Copt-cquere/src/pyzx/circuits/benchmarking_circuits/Fast/before/calum/arithmetic_and_toffoli/tof_4.qasm").to_basic_gates()
res_circ = flow_opt(circ)
print(res_circ.stats())
print(circ.stats())

Circuit  on 7 qubits with 57 gates.
        23 is the T-count
        34 Cliffords among which
        24 2-qubit gates (19 CNOT, 5 other) and
        10 Hadamard gates.
Circuit tof_4.qasm on 7 qubits with 95 gates.
        35 is the T-count
        60 Cliffords among which
        30 2-qubit gates (30 CNOT, 0 other) and
        30 Hadamard gates.


In [2]:
import os
from collections import Counter
import pandas as pd

def parse_qasm(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        
    gate_counter = Counter()
    qubits = set()
    total_gates = 0

    for line in lines:
        if line.startswith('OPENQASM') or line.startswith('include') or line.startswith('creg') or line.startswith('measure') or line.startswith('if'):
            continue
        elif line.startswith("qreg"):
            num_qubits = int(line.split('[')[1].split(']')[0])
            qubits.update(range(num_qubits))
            continue
        
        gate = line.split(' ')[0]
        gate_counter[gate] += 1
        total_gates += 1

    return gate_counter, total_gates, len(qubits)

def extract_statistics(directory):
    stats = []
    for filename in os.listdir(directory):
        if filename.endswith('.qasm'):
            file_path = os.path.join(directory, filename)
            file_stats, total_gates, qubits = parse_qasm(file_path)
            stats.append((filename, qubits, total_gates, file_stats))
    return stats
def create_dataframe(stats):
    data = []
    for stat in stats:
        filename, qubits, total_gates, gate_counter = stat
        row = {
            'Filename': filename,
            'Qubits': qubits,
            'Total Gates': total_gates
        }
        row.update({gate: count for gate, count in gate_counter.items()})
        data.append(row)
    
    df = pd.DataFrame(data).fillna('-')
    
    # Add percentage columns
    for gate in ['x', 'h', 'ccx', 'cx']:
        df[f'{gate}(%)'] = df.apply(lambda row: (row[gate] / row['Total Gates'] * 100) if row[gate] != '-' else '-', axis=1)
    
    return df

# Example usage
directory = '/home/jnogue/qilimanjaro/Copt-cquere/src/pyzx/circuits/benchmarking_circuits/Fast/before/calum/arithmetic_and_toffoli'
stats = extract_statistics(directory)
df = create_dataframe(stats)
df


,Filename,Qubits,Total Gates,h,ccx,cx,x,x(%),h(%),ccx(%),cx(%)
0,gf2^6_mult.qasm,18,135,94,36,5.0,-,-,69.629630,26.666667,3.703704
1,barenco_tof_10.qasm,19,130,98,32,-,-,-,75.384615,24.615385,-
2,barenco_tof_4.qasm,7,34,26,8,-,-,-,76.470588,23.529412,-
3,mod5_4.qasm,5,23,14,4,4.0,1.0,4.347826,60.869565,17.391304,17.391304
4,qcla_com_7.qasm,24,153,97,29,12.0,15.0,9.803922,63.398693,18.954248,7.843137
5,gf2^5_mult.qasm,15,97,68,25,4.0,-,-,70.103093,25.773196,4.123711
6,csum_mux_9.qasm,30,140,84,28,-,28.0,20.0,60.000000,20.000000,-
7,tof_5.qasm,9,35,28,7,-,-,-,80.000000,20.000000,-
8,qcla_mod_7.qasm,26,294,200,59,28.0,7.0,2.380952,68.027211,20.068027,9.52381
9,gf2^8_mult.qasm,24,243,158,64,21.0,-,-,65.020576,26.337449,8.641975


In [23]:
import numpy as np
import random
import json

target_percentages = {
    'h': [element if element != '-' else 0 for element in list(df["h(%)"])],
    'ccx': [element if element != '-' else 0 for element in list(df["ccx(%)"])],
    'cx': [element if element != '-' else 0 for element in list(df["cx(%)"])],
    'x': [element if element != '-' else 0 for element in list(df["x(%)"])]
}

def calculate_cdf(target_percentages):
    cdfs = {}
    for gate, percentages in target_percentages.items():
        cdf = np.cumsum(percentages) / np.sum(percentages)
        cdfs[gate] = cdf
    return cdfs


cdfs = calculate_cdf(target_percentages)

with open('cdfs.json', 'w') as file:
    json.dump({key: list(value) for key, value in cdfs.items()}, file)
print("CDFs:", cdfs)
print(target_percentages["x"])

CDFs: {'h': array([0.04732325, 0.09855783, 0.15053048, 0.19190001, 0.23498845,
       0.28263348, 0.32341203, 0.37778342, 0.42401759, 0.46820833,
       0.50143529, 0.54139002, 0.58022673, 0.62100527, 0.66128038,
       0.71565177, 0.76374953, 0.81812092, 0.86977375, 0.91408192,
       0.95291863, 1.        ]), 'ccx': array([0.05900788, 0.1134767 , 0.16554248, 0.20402588, 0.24596776,
       0.30299857, 0.34725448, 0.39151039, 0.43591684, 0.49419623,
       0.52124151, 0.55946252, 0.59107389, 0.6353298 , 0.67016084,
       0.71441675, 0.76888557, 0.81314148, 0.86624857, 0.9078149 ,
       0.93942627, 1.        ]), 'cx': array([0.01958448, 0.01958448, 0.01958448, 0.11154639, 0.15301941,
       0.17482481, 0.17482481, 0.17482481, 0.2251849 , 0.27088202,
       0.42951632, 0.53687488, 0.68795516, 0.79371136, 0.80839972,
       0.80839972, 0.83280499, 0.83280499, 0.83280499, 0.91752681,
       0.9822755 , 1.        ]), 'x': array([0.        , 0.        , 0.        , 0.04962892, 0.16153727,


In [33]:
import os
from qiskit import QuantumCircuit

# Directory containing QASM files
directory = '/home/jnogue/qilimanjaro/Copt-cquere/src/pyzx/circuits/benchmarking_circuits/Fast/before/calum/arithmetic_and_toffoli'

def get_circuit_info(directory):
    circuit_info = {}
    for filename in os.listdir(directory):
        if filename.endswith('.qasm'):
            filepath = os.path.join(directory, filename)
            qc = QuantumCircuit.from_qasm_file(filepath)
            num_qubits = qc.num_qubits
            depth = qc.depth()
            circuit_info[filename] = {'num_qubits': num_qubits, 'depth': depth}
    return circuit_info

# Retrieve information for all QASM files in the directory
info = get_circuit_info(directory)

# Print the dictionary
print(info)


{'gf2^6_mult.qasm': {'num_qubits': 18, 'depth': 47}, 'barenco_tof_10.qasm': {'num_qubits': 19, 'depth': 98}, 'barenco_tof_4.qasm': {'num_qubits': 7, 'depth': 26}, 'mod5_4.qasm': {'num_qubits': 5, 'depth': 23}, 'qcla_com_7.qasm': {'num_qubits': 24, 'depth': 23}, 'gf2^5_mult.qasm': {'num_qubits': 15, 'depth': 38}, 'csum_mux_9.qasm': {'num_qubits': 30, 'depth': 22}, 'tof_5.qasm': {'num_qubits': 9, 'depth': 23}, 'qcla_mod_7.qasm': {'num_qubits': 26, 'depth': 58}, 'gf2^8_mult.qasm': {'num_qubits': 24, 'depth': 69}, 'rc_adder_6.qasm': {'num_qubits': 14, 'depth': 40}, 'adder_8.qasm': {'num_qubits': 24, 'depth': 78}, 'csla_mux_3.qasm': {'num_qubits': 15, 'depth': 24}, 'vbe_adder_3.qasm': {'num_qubits': 10, 'depth': 28}, 'mod_red_21.qasm': {'num_qubits': 11, 'depth': 58}, 'tof_4.qasm': {'num_qubits': 7, 'depth': 17}, 'gf2^4_mult.qasm': {'num_qubits': 12, 'depth': 31}, 'tof_10.qasm': {'num_qubits': 19, 'depth': 53}, 'barenco_tof_5.qasm': {'num_qubits': 9, 'depth': 38}, 'qcla_adder_10.qasm': {'nu

In [29]:
import numpy as np
import random

def sample_gate(cdfs):
    gates = list(cdfs.keys())
    samples = []

    for gate in gates:
        r = random.random()
        idx = np.searchsorted(cdfs[gate], r)
        samples.append(cdfs[gate][idx])

    # Normalize 
    total = sum(samples)
    normalized_samples = [sample / total for sample in samples]
    normalized_samples_dict = {key:samples for key,samples in zip(gates,normalized_samples)}
    return normalized_samples_dict



samples= sample_gate(cdfs)
print("Sampled", samples)


Sampled {'h': 0.16071956643259588, 'ccx': 0.1854511392341054, 'cx': 0.2284014974914574, 'x': 0.42542779684184134}
